## NOTEBOOK TRIALS


### IMPORTING LIBRARIES

In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

### Creating PINECONE VECTORDB

In [2]:
PINECONE_API_KEY = "4462428f-e1a5-44dd-82cc-788b6a1f963c"
PINECONE_API_ENV = "gcp-starter"

### Extract PDF Data

In [3]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [4]:
extracted_data = load_pdf("data")

In [5]:
extracted_data

[Document(page_content='INTRODUCTION\nTO\nMACHINE LEARNING\nAN EARLY DRAFT OF A PROPOSED\nTEXTBOOK\nNils J. Nilsson\nRobotics Laboratory\nDepartment of Computer Science\nStanford University\nStanford, CA 94305\ne-mail: nilsson@cs.stanford.edu\nNovember 3, 1998\nCopyright c\r2005 Nils J. Nilsson\nThis material may not be copied, reproduced, or distributed without the\nwritten permission of the copyright holder.', metadata={'source': 'data\\pdf1.pdf', 'page': 0}),
 Document(page_content='ii', metadata={'source': 'data\\pdf1.pdf', 'page': 1}),
 Document(page_content='Contents\n1 Preliminaries 1\n1.1 Introduction . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 1\n1.1.1 What is Machine Learning? . . . . . . . . . . . . . . . . . 1\n1.1.2 Wellsprings of Machine Learning . . . . . . . . . . . . . . 3\n1.1.3 Varieties of Machine Learning . . . . . . . . . . . . . . . . 4\n1.2 Learning Input-Output Functions . . . . . . . . . . . . . . . . . . 5\n1.2.1 Types of Learning . . . . . . 

### Create TEXT CHUNKS

In [6]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 1301


### Use of EMBEDDING MODEL

In [8]:
# Use your embedding model name 

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()

e:\END TO END LLAMA@ CHATBOT\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# Model Details
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [12]:
## Testing our embedding model
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [14]:
# Vector Representation of "Hello World"
# Dimension of vector : 384
query_result

[-0.03447726368904114,
 0.03102320246398449,
 0.006734968163073063,
 0.026108931750059128,
 -0.03936202451586723,
 -0.16030243039131165,
 0.06692398339509964,
 -0.006441442761570215,
 -0.04745052754878998,
 0.014758856035768986,
 0.07087531685829163,
 0.05552757903933525,
 0.019193340092897415,
 -0.026251325383782387,
 -0.010109522379934788,
 -0.02694045566022396,
 0.02230742760002613,
 -0.022226622328162193,
 -0.14969255030155182,
 -0.0174929890781641,
 0.007676256354898214,
 0.05435231328010559,
 0.0032544289715588093,
 0.03172588720917702,
 -0.08462140709161758,
 -0.029405998066067696,
 0.05159555748105049,
 0.04812406003475189,
 -0.003314817091450095,
 -0.05827919766306877,
 0.04196927323937416,
 0.022210652008652687,
 0.128188818693161,
 -0.02233898639678955,
 -0.011656259186565876,
 0.06292835623025894,
 -0.03287637606263161,
 -0.0912260189652443,
 -0.031175369396805763,
 0.05269956216216087,
 0.04703478515148163,
 -0.0842030867934227,
 -0.030056225135922432,
 -0.0207448527216911

### Setting Up PINECONE

In [22]:
#Initializing the Pinecone
import pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="medical-chatbot"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

ForbiddenException: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': '418fdd5dc542f1e17f03bd5da8cb5659', 'Date': 'Wed, 10 Apr 2024 13:23:43 GMT', 'Server': 'Google Frontend', 'Content-Length': '142', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"FORBIDDEN","message":"Request failed. Your free plan supports 1 starter indexes. Use a different index type."},"status":403}
